In [1]:
import itk
from itkwidgets import view, compare, checkerboard
import ipywidgets
import os

**Image registration** finds the *spatial transformation that aligns images in the presence of noise*.
First, let's load our **fixed image** and the image we will align to our fixed image, the **moving image**.

Fixed image:

In [2]:
# these FileUpload widgets have maximum size
# controlled via jupyter_notebook_config.py
# https://github.com/jupyter-widgets/ipywidgets/issues/2522
fixedUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(fixedUploader)

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

Moving image:

In [3]:
movingUploader = ipywidgets.FileUpload(
    accept='image/*,.nrrd,.mha,.nii',
    multiple=False
)
display(movingUploader)

FileUpload(value={}, accept='image/*,.nrrd,.mha,.nii', description='Upload')

In [4]:
viewButton = ipywidgets.Button(description="Load and view")
display(viewButton)

appState={'fixed':None, 'moving':None}

def viewImages(b):
    # write to temporary image file
    # I don't know how to read image from memory in Python
    # using FileUpload's syntax for ipywidgets <= 7
    tempFilename="./fixed_"+ next(iter(fixedUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(fixedUploader.data[0])
    appState['fixed'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)

    tempFilename="./moving_"+ next(iter(movingUploader.value))
    with open(tempFilename, "wb") as fp:
        fp.write(movingUploader.data[0])
    appState['moving'] = itk.imread(tempFilename, itk.F)
    os.remove(tempFilename)
    
    display(compare(appState['fixed'], appState['moving'], link_cmap=True, link_gradient_opacity=True))

viewButton.on_click(viewImages)

Button(description='Load and view', style=ButtonStyle())

Before registration, the moving image is not aligned with the fixed image.

In [5]:
checkerboardButton = ipywidgets.Button(description="View checkerboard")
display(checkerboardButton)

def checkerboardImages(b):  
    display(checkerboard(appState['fixed'], appState['moving'], ui_collapsed=False))

checkerboardButton.on_click(checkerboardImages)

Button(description='View checkerboard', style=ButtonStyle())

In [6]:
regType=ipywidgets.RadioButtons(
    options=['rigid', 'affine', 'bspline'],
    value='rigid', # Defaults to 'pineapple'
    description='Type:',
    disabled=False
)
display(regType)

registerButton = ipywidgets.Button(description="Register and view")
display(registerButton)

out = ipywidgets.Output()
display(out)

def registerImages(b):
    out.clear_output(wait=True)
    parameters = itk.ParameterObject.New()

    resolutions = 3
    default_rigid = parameters.GetDefaultParameterMap("rigid", resolutions)
    parameters.AddParameterMap(default_rigid)

    if (regType.value!='rigid'):
        resolutions = 2
        default_affine = parameters.GetDefaultParameterMap("affine", resolutions)
        parameters.AddParameterMap(default_affine)

    if (regType.value=='bspline'):
        resolutions = 1
        default_bspline = parameters.GetDefaultParameterMap("bspline", resolutions)
        parameters.AddParameterMap(default_bspline)

    parameters.RemoveParameter("ResultImageFormat")
    with out:
        print('Executing '+regType.value+' registration. Please wait...')
    
    registered_moving = itk.elastix_registration_method(appState['fixed'], appState['moving'], parameter_object=parameters)
    display(checkerboard(appState['fixed'], registered_moving))
    out.clear_output()

registerButton.on_click(registerImages)

RadioButtons(description='Type:', options=('rigid', 'affine', 'bspline'), value='rigid')

Button(description='Register and view', style=ButtonStyle())

Output()